In [70]:
import numpy as np
from casadi import *
import casadi as cs
import matplotlib.pyplot as plt
import seaborn as sns
from util import *
import itertools 
from time import perf_counter
import sympy as sym
from IPython.display import display, Math
import util

In [59]:
# from dpilqr import util

In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
theta_max = np.pi / 6
phi_max = np.pi / 6

v_max = 3
v_min = -3

theta_min = -np.pi / 6
phi_min = -np.pi / 6

tau_max = 15
tau_min = 0

x_min = -5
x_max = 5

y_min = -5
y_max = 5

z_min = 0
z_max = 3.5

max_input_base = np.array([[theta_max], [phi_max], [tau_max]])
min_input_base = np.array([[theta_min], [phi_min], [tau_min]])
max_state_base = np.array([[x_max], [y_max], [z_max], [v_max],[v_max], [v_max]])
min_state_base = np.array([[x_min], [y_min], [z_min], [v_min],[v_min],[v_min]])


In [140]:
n_agents = 3

In [141]:

max_input = np.tile(max_input_base,(n_agents,1))
min_input = np.tile(min_input_base,n_agents)
max_state = np.tile(max_state_base,n_agents)
min_state = np.tile(min_state_base,n_agents)

In [142]:
max_input

array([[ 0.524],
       [ 0.524],
       [15.   ],
       [ 0.524],
       [ 0.524],
       [15.   ],
       [ 0.524],
       [ 0.524],
       [15.   ]])

In [143]:
max_input = np.array([[theta_max], [phi_max], [tau_max], \
                          [theta_max], [phi_max], [tau_max], \
                          [theta_max], [phi_max], [tau_max]])

In [146]:
max_input.shape

(9, 1)

In [86]:
x0 = np.random.rand(18,1)
xf = np.random.rand(18,1)
x_dims = [6]*3
n_dims = [3,3,3]
radius = 0.5
ids =  [100 + i for i in range(n_agents)]
g = 9.81
u_dims = [3]*3
N = 10

In [87]:
u_ref = np.array([0,0,g,0,0,g,0,0,g])

In [88]:
# rel_dists = util.compute_pairwise_distance(x0, x_dims, n_d=3)
rel_dists = util.compute_pairwise_distance_nd_Sym(x0,x_dims,n_dims)
#Pairwise distance between any agent and a human agent is the 2-D planar Euclidean distance
#since the human agent is assumed to be a cylindrical object with constant height

#How do I know if the current subproblem has a human agent?
# ids[-n_humans:]: id(s) of the human agents
graph = util.define_inter_graph_threshold(x0, radius, x_dims, ids, n_dims)

# print(
    # f"current interaction graph is {graph}, the pairwise distances between each agent is {rel_dists} \n"
# )
# x0 is updated until convergence (treat x0 as the combined CURRENT state)

# break up the problem into potential-game sub-problems at every outer iteration
split_problem_states_initial = split_graph(x0.T, x_dims, graph)
# print(split_problem_states_initial)
split_problem_states = split_graph(xf.T, x_dims, graph)
split_problem_inputs = split_graph(u_ref.reshape(-1, 1).T, u_dims, graph)

# Initiate different instances of Opti() object
# Each Opti() object corresponds to a subproblem (there is NO central node)
# Note that when 2 agents are combined into a single problem, we have 2 copies of the same sub problem
########################################################################
# Setting up the solvers:
d = {}
states = {}
inputs = {}
cost_fun_list = []

d = {}  # dictionary holding Opti() objects (or subproblems)
states = {}  # dictionary holding symbolic state trajectory for each sub-problem
inputs = {}  ##dictionary holding symbolic input trajectory for each sub-problem

for i, j in enumerate(split_problem_states_initial):
    d["opti_{0}".format(i)] = cs.Opti()
    states["X_{0}".format(i)] = d[f"opti_{i}"].variable(j.shape[1], N + 1)

for i, j in enumerate(split_problem_inputs):
    inputs["U_{0}".format(i)] = d[f"opti_{i}"].variable(j.shape[1], N)

# Storing objective functions for each sub-problem into a list:
for i in range(len(split_problem_states_initial)):
    cost_fun_list.append(
        objective(
            states[f"X_{i}"],
            inputs[f"U_{i}"],
            split_problem_inputs[i].reshape(
                -1,
            ),
            split_problem_states[i].reshape(-1, 1),
            np.eye(split_problem_states_initial[i].shape[1]) * 100,
            np.eye(
                split_problem_inputs[i]
                .reshape(
                    -1,
                )
                .shape[0]
            )
            * 0.1,
            np.eye(split_problem_states_initial[i].shape[1]) * 1000,
        )
    )

determining interaction graph with the following pair-wise distance : [[0.321 0.481 0.426]]


In [92]:
n_inputs = 3
n_states = 6

In [93]:
min_max_input_list = generate_min_max_input(inputs, n_inputs,theta_max,
                  theta_min,tau_max,tau_min,phi_max,phi_min)
min_max_state_list = generate_min_max_state(states, n_states,x_min,
                  x_max,y_min,y_max,z_min,z_max,v_min,v_max)

In [94]:
min_max_input_list

[(array([[-0.524, -0.524,  0.   , -0.524, -0.524,  0.   , -0.524, -0.524,
           0.   ]]),
  array([[ 0.524,  0.524, 15.   ,  0.524,  0.524, 15.   ,  0.524,  0.524,
          15.   ]])),
 (array([[-0.524, -0.524,  0.   , -0.524, -0.524,  0.   , -0.524, -0.524,
           0.   ]]),
  array([[ 0.524,  0.524, 15.   ,  0.524,  0.524, 15.   ,  0.524,  0.524,
          15.   ]])),
 (array([[-0.524, -0.524,  0.   , -0.524, -0.524,  0.   , -0.524, -0.524,
           0.   ]]),
  array([[ 0.524,  0.524, 15.   ,  0.524,  0.524, 15.   ,  0.524,  0.524,
          15.   ]]))]

In [129]:
id_humans = ids[-1:]

In [145]:
for (
    di,
    statesi,
    inputsi,
    costi,
    state_boundsi,
    input_boundsi,
    (prob, ids_),
    count,
) in zip(
    d.values(),
    states.values(),
    inputs.values(),
    cost_fun_list,
    min_max_state_list,
    min_max_input_list,
    graph.items(),
    range(len(d)),
):  # loop over sub-problems
#     min_states, max_states = state_boundsi
#     min_inputs, max_inputs = input_boundsi
#     n_states_local = statesi.shape[0]
#     n_inputs_local = inputsi.shape[0]
#     x_dims_local = [int(n_states)] * int(n_states_local / n_states)
#     drones_count = len(x_dims_local)
#     for k in range(N):  # loop over control intervals
               
#         if drones_count !=0:
#             # print(f'inputsi has shape{inputsi[0:drones_count*n_inputs,k].shape},max_inputs has shape{max_inputs.shape}\n')
#             di.subject_to(inputsi[0:drones_count*n_inputs,k] <= max_inputs[: , 0:drones_count*n_inputs].T)
#             di.subject_to(min_inputs[: , 0:drones_count*n_inputs].T <= inputsi[0:drones_count*n_inputs,k])
#             # print(inputsi[0:9,0].shape)
#             # print(drones_count*n_inputs)
#             # print(max_inputs.T.shape)

    # print(prob,ids_)
    human_count = sum(item in ids_ for item in id_humans)
    drones_count = len(x_dims_local)-human_count
    print(drones_count)
    n_dims_local = [3]*drones_count 
    n_dims_local+= [2]*human_count
    print(f'n_dims is {n_dims_local}')

2
n_dims is [3, 3, 2]
2
n_dims is [3, 3, 2]
2
n_dims is [3, 3, 2]


In [101]:
graph

{100: [100, 101, 102], 101: [100, 101, 102], 102: [100, 101, 102]}

In [96]:
min_states

array([[-5, -5,  0, -3, -3, -3, -5, -5,  0, -3, -3, -3, -5, -5,  0, -3,
        -3, -3]])

In [98]:
max_states

array([[5. , 5. , 3.5, 3. , 3. , 3. , 5. , 5. , 3.5, 3. , 3. , 3. , 5. ,
        5. , 3.5, 3. , 3. , 3. ]])

In [99]:
min_inputs

array([[-0.524, -0.524,  0.   , -0.524, -0.524,  0.   , -0.524, -0.524,
         0.   ]])

In [100]:
max_inputs

array([[ 0.524,  0.524, 15.   ,  0.524,  0.524, 15.   ,  0.524,  0.524,
        15.   ]])